In [1]:
!nvidia-smi

Thu Nov 30 00:57:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0    29W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip3 install transformers==4.33.2
!pip3 install optimum==1.13.2
!pip3 install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [4]:
model_name_or_path = 'TheBloke/Llama-2-13B-German-Assistant-v4-GPTQ'  # "TheBloke/Llama-2-13B-chat-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map='auto',
                                             trust_remote_code=False,
                                             revision="gptq-4bit-32g-actorder_True")
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

In [5]:
import pickle

with open("definitions.pickle", "rb") as file:
    definitions = pickle.load(file)
with open("sentences.pickle", "rb") as file:
    sentences = pickle.load(file)

definitions["10"]

'Die "Negation:NEG_XgeschweigedennY-Konstruktion" korreliert zwei Propositionen, die jeweils einen Punkt auf einem pragmatisch definiertem Maßstab markieren. Die erste Proposition ist pragmatisch stärker als die zweite Proposition; so zieht, pragmatisch betrachtet, die Wahrheit der ersten Proposition automatisch die Wahrheit der zweiten Proposition nach sich. Einfacher ausgedrückt: Ist die erste Proposition wahr, so muss die zweite Proposition ebenfalls wahr sein. Die erste Proposition ist darüber hinaus auch informativer als die zweite Proposition, da die erstgenannte pragmatisch betrachtet die letztgenannte einschließt, während diese wiederum die diskursrelevantere der beiden Propositionen ist. Die Konstruktion umfasst das konstruktionsevozierende Element (KEE) "geschweige_denn", die internen Kern-Konstruktionselemente (Kern-KE) "Erstes_Konjunkt" und "Zweites_Konjunkt" sowie die externen Kern-KE "Negator" und "Fokuskontext". Das "Erste_Konjunkt" geht dem "KEE" voraus, auf welches das

In [6]:
sentences[10]

{'""Es ist mehr als klar , dass das ganze Unternehmen keine drei Euro wert ist - geschweige denn die drei Euro je Aktie , die Flowers verlangt "" , sagte ein Börsianer .',
 '""Ich kann mich kaum noch bücken , ich kann meine Laptoptasche nicht mehr tragen , geschweige denn meine Tochter .""',
 'Abermillionen rings um die Welt , die das Spektakel auf dem Schirm beobachtet haben , kennen nicht einmal die Namen der Staatschefs von China , Indien oder den europäischen Ländern , geschweige denn , dass sie ihre Amtseinführung bestaunen würden .',
 'Bei ihnen ist es bereits schwer vorstellbar , dass sie Geiseln nehmen , geschweige denn diese ermorden , und schon gar keine Frauen oder sogar Kinder .""',
 'Bis heute sind viele dieser Untaten , dieser Massaker an der »Heimatfront« , kaum aufgeklärt , geschweige denn im öffentlichen Gedächtnis präsent .',
 'Bush selbst hat wahrscheinlich kaum einmal den Namen Johann Strauss gehört , geschweige denn eine Seite von Leo gelesen , doch wurde seine Pol

In [ ]:
import re
import os
from tqdm.notebook import tqdm

def generate_examples(definition: str, sentence: str, temperature=0.75, top_p=0.95, top_k=1, max_new_tokens=1024):
    prompt = lambda definition, sentence: f'''### User: Du bist kreativ und gewissenhaft. Hier ist eine Definition: {definition} Bilde neue Sätze gemäß dieser Definition. Gib die Sätze in einer Python-Liste aus. Gib sonst nichts aus.
    ### Example: {sentence}
    ### Assistant:["'''

    prompt_length = len(prompt(definition, sentence))
    output = []
    first_output = ""
    i = 100  # only try 100 times
    while (
        (not any([c.isalpha() for c in first_output]))
        or any([x in first_output for x in {"Konstruktion", "Satz", "Überschrift", "_", ":", "XY", "XP", "X ", "Y ", "X.", "Y."}])
        or re.search(r".*\].*\[.*", first_output)
    ):
        input_ids = tokenizer(prompt(definition, sentence), return_tensors='pt').input_ids.cuda()
        output = model.generate(inputs=input_ids, temperature=temperature,
                                do_sample=True, top_p=top_p, top_k=top_k,
                                max_new_tokens=max_new_tokens)
        output = tokenizer.decode(output[0])[prompt_length:].strip()
        output = re.findall('\[.*\]', output)
        if len(output) > 0:
            first_output = output[0]
        i -= 1
        print(i, end=" ")
        if i == 0:
            first_output = "[]"
            break
        #print(f"\t{output}")
    #print(f"\n{output}")
    print()
    return first_output

examples = []
if os.path.exists("examples.pickle"):
    with open("examples.pickle", "rb") as file:
        examples = pickle.load(file)
for k in tqdm(list(definitions.keys())[len(examples):]):
    definition = definitions[str(k)]
    sentence = str(list(sentences[int(k)])[0:3])  # get some sentences

    example = generate_examples(definition, sentence, temperature=0.75, max_new_tokens=1000,
                                top_k=100, top_p=0.99)
    #example = re.findall('\[("[^"]*"(?:, ?"[^"]*")*)\]', example)[0]
    try:
        print((sentence, example))
        examples.append((sentence, example))  # one is enough
    except:
        print((sentence, "[]"))
        examples.append((sentence, "[]"))

    with open("examples.pickle", "wb") as file:
        pickle.dump(examples, file)

  0%|          | 0/211 [00:00<?, ?it/s]

99 98 97 
("['»Ohne Europa sind viele Fragen nicht mehr seriös in Angriff zu nehmen , geschweige denn zu lösen« , meint der grüne Bundesparteisekretär Lothar Lockl .', 'Bis heute sind viele dieser Untaten , dieser Massaker an der »Heimatfront« , kaum aufgeklärt , geschweige denn im öffentlichen Gedächtnis präsent .', 'Plötzlich konnte Quelle keine Rechnung mehr bezahlen , geschweige denn den Hauptkatalog für den Herbst und Winter finanzieren .']", '["Ich habe heute eine Socke verloren , geschweige denn den ganzen Tag", "Das Auto, das ich gekauft habe, war zu teuer , geschweige denn zu ungerechtfertigt", "Ich habe gestern viel gegessen , geschweige denn zu viel"]')
99 98 97 
("['Sicherheitsgefühl und objektive Sicherheit seien für die Lebensqualität gleich wichtig .', 'Wir haben schweizweit eine Patt - Situation , weil der linke und der rechte Block gleich stark sind .', 'Bis auf wenige Millimeter sind die Geräte praktisch gleich groß .']", '["Die äquative Pluralkonstruktion zeigt die G

In [ ]:
while True:pass